In [3]:
import pandas as pd
import numpy as np

# Load the Excel file
excel_file_path = r"F:\PythonProgram\Health_Inssurance\Health_insurance_dummy_data.xlsx"
df = pd.read_excel(excel_file_path)

# Function to flag extreme values
def flag_extreme_values(df, column, threshold=2):
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    extreme_values = z_scores > threshold
    return extreme_values

# Flag extreme values for Income and Assets based on Profession, Age, and Location
for column in ['Income', 'Assets']:
    for category in ['Profession', 'Age', 'Location']:
        mask = df.groupby(category).apply(lambda x: flag_extreme_values(x, column))
        df[f'{column}_Extreme_{category}'] = mask.reset_index(level=0, drop=True)

# Identify applicants not suitable for insurance based on extreme values
unsuitable_applicants = df[(df['Income_Extreme_Profession'] | df['Income_Extreme_Age'] | df['Income_Extreme_Location'] |
                            df['Assets_Extreme_Profession'] | df['Assets_Extreme_Age'] | df['Assets_Extreme_Location'])]

# Display the identified applicants
print("Applicants not suitable for insurance:")
print(unsuitable_applicants[['Application ID', 'Name']])


Applicants not suitable for insurance:
Empty DataFrame
Columns: [Application ID, Name]
Index: []


In [7]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import PatternFill

# Load the Excel file
excel_file_path = r"F:\PythonProgram\Health_Inssurance\Health_insurance_dummy_data.xlsx"
df = pd.read_excel(excel_file_path)

# Function to flag extreme values
def flag_extreme_values(df, column, threshold=2):
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    extreme_values = z_scores > threshold
    return extreme_values

# Flag extreme values for Income and Assets based on Profession, Age, and Location
for column in ['Income', 'Assets']:
    for category in ['Profession', 'Age', 'Location']:
        mask = df.groupby(category).apply(lambda x: flag_extreme_values(x, column))
        df[f'{column}_Extreme_{category}'] = mask.reset_index(level=0, drop=True)

# Identify and flag doubtful applicants
doubtful_applicants = df[(df['Income_Extreme_Profession'] | df['Income_Extreme_Age'] | df['Income_Extreme_Location'] |
                           df['Assets_Extreme_Profession'] | df['Assets_Extreme_Age'] | df['Assets_Extreme_Location'])]

# Save the results to a new Excel file with highlighted doubtful applicants
output_excel_file_path = 'doubtful_applicants_output.xlsx'
with pd.ExcelWriter("F:\PythonProgram\Health_Inssurance\doubtful_applicants_output_1.xlsx", engine='openpyxl') as writer:
    writer.book = openpyxl.load_workbook(excel_file_path)
    doubtful_applicants.to_excel(writer, sheet_name='Doubtful_Applicants', index=False)

    # Highlight doubtful applicants in the output sheet
    output_sheet = writer.sheets['Doubtful_Applicants']
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Extreme' in cell.column_letter:
                cell.fill = fill

print(f"Doubtful applicants flagged and saved to {F:\PythonProgram\Health_Inssurance\doubtful_applicants_output_1.xlsx}")


IndexError: At least one sheet must be visible

In [13]:
pip install --upgrade pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill

# Read the health insurance data
file_path = 'Health_Insu_data.xlsx'
df = pd.read_excel(file_path)

# Function to flag extreme values
def flag_extreme_values(series, threshold=2):
    z_scores = (series - series.mean()) / series.std()
    return abs(z_scores) > threshold

# Define keywords for categorizing professions into 'White Collar' and 'Blue Collar'
white_collar_keywords = ['manager', 'engineer', 'analyst']
blue_collar_keywords = ['driver', 'technician', 'operator']

# Categorize professions into 'White Collar' and 'Blue Collar'
df['Profession_Category'] = 'Other'
df.loc[df['Profession'].str.lower().isin(white_collar_keywords), 'Profession_Category'] = 'White Collar'
df.loc[df['Profession'].str.lower().isin(blue_collar_keywords), 'Profession_Category'] = 'Blue Collar'

# Define conditions for extreme values, age, and location
conditions = [
    # Condition for extreme values in income and assets within each profession category
    (df['Income'].groupby(df['Profession_Category']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Profession_Category']).transform(flag_extreme_values)),
    
    # Condition for extreme values in income and assets within each age group
    (df['Income'].groupby(df['Age']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Age']).transform(flag_extreme_values)),
    
    # Condition for extreme values in income and assets within each location
    (df['Income'].groupby(df['Location']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Location']).transform(flag_extreme_values)),
    
    # Custom conditions for age and location
    ((df['Age'] >= 30) & (df['Age'] <= 50)),
    (df['Location'] == 'YourLocation'),
]

# Combine conditions with OR logic
extreme_values_condition = pd.concat(conditions, axis=1).any(axis=1)

# Create a new column 'Doubtful_Applicant' to flag doubtful applicants
df['Doubtful_Applicant'] = False
df.loc[extreme_values_condition, 'Doubtful_Applicant'] = True

# Save the results to a new Excel file with highlighted doubtful applicants
output_excel_file_path = 'Doubtful_Applicants_Output3.xlsx'

# Create a new workbook
output_workbook = Workbook()

# Write the data to the new workbook
df.to_excel(output_workbook, index=False, sheet_name='Doubtful_Applicants')

# Highlight doubtful applicants in the output sheet
output_sheet = output_workbook['Doubtful_Applicants']
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
for col in output_sheet.iter_cols():
    for cell in col:
        if 'Doubtful_Applicant' in cell.column_letter:
            cell.fill = fill

# Save the new workbook
output_workbook.save(output_excel_file_path)

print(f"Doubtful applicants flagged and saved to {output_excel_file_path}")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Kirtibala.Gujar\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Kirtibala.Gujar\AppData\Local\Temp\ipykernel_10984\3925820962.py", line 56, in <module>
    df.to_excel(output_workbook, index=False, sheet_name='Doubtful_Applicants')
  File "C:\Users\Kirtibala.Gujar\AppData\Local\anaconda3\Lib\site-packages\pandas\core\generic.py", line 2252, in to_excel
    as_json = data.to_json(orient="table")
  File "C:\Users\Kirtibala.Gujar\AppData\Local\anaconda3\Lib\site-packages\pandas\io\formats\excel.py", line 943, in write
    writer = ExcelWriter(
             ^^^^^^^^^^^^
  File "C:\Users\Kirtibala.Gujar\AppData\Local\anaconda3\Lib\site-packages\pandas\io\excel\_openpyxl.py", line 60, in __init__
  File "C:\Users\Kirtibala.Gujar\AppData\Local\anaconda3\Lib\site-packages\pandas\io\excel\_base.py", line 1219, in __init__


In [22]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the health insurance data
file_path = 'Health_Insu_data.xlsx'
df = pd.read_excel(file_path)

# Function to flag extreme values
def flag_extreme_values(series, threshold=2):
    z_scores = (series - series.mean()) / series.std()
    return abs(z_scores) > threshold

# Define keywords for categorizing professions into 'White Collar' and 'Blue Collar'
white_collar_keywords = ['manager', 'engineer', 'analyst']
blue_collar_keywords = ['driver', 'technician', 'operator']

# Categorize professions into 'White Collar' and 'Blue Collar'
df['Profession_Category'] = 'Other'
df.loc[df['Profession'].str.lower().isin(white_collar_keywords), 'Profession_Category'] = 'White Collar'
df.loc[df['Profession'].str.lower().isin(blue_collar_keywords), 'Profession_Category'] = 'Blue Collar'

# Define conditions for extreme values, age, and location
conditions = [
    # Condition for extreme values in income and assets within each profession category
    (df['Income'].groupby(df['Profession_Category']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Profession_Category']).transform(flag_extreme_values)),
    
    # Condition for extreme values in income and assets within each age group
    (df['Income'].groupby(df['Age']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Age']).transform(flag_extreme_values)),
    
    # Condition for extreme values in income and assets within each location
    (df['Income'].groupby(df['Location']).transform(flag_extreme_values)),
    (df['Assets'].groupby(df['Location']).transform(flag_extreme_values)),
    
    # Custom conditions for age and location
    ((df['Age'] >= 30) & (df['Age'] <= 50)),
    (df['Location'] == 'YourLocation'),
]

# Combine conditions with OR logic
extreme_values_condition = pd.concat(conditions, axis=1).any(axis=1)

# Create a new column 'Doubtful_Applicant' to flag doubtful applicants
df['Doubtful_Applicant'] = False
df.loc[extreme_values_condition, 'Doubtful_Applicant'] = True

# Save the results to a new Excel file with highlighted doubtful applicants
output_excel_file_path = 'Doubtful_Applicants_Output3.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Doubtful_Applicants')

    # Highlight doubtful applicants in the output sheet
    output_sheet = writer.sheets['Doubtful_Applicants']
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Doubtful_Applicant' in cell.column_letter:
                cell.fill = fill

print(f"Doubtful applicants flagged and saved to {output_excel_file_path}")


Doubtful applicants flagged and saved to Doubtful_Applicants_Output3.xlsx


In [7]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the life insurance data
file_path = 'Life_Insurance_data.xlsx'
df = pd.read_excel(file_path)

# Define the conditions for mismatched applicants
mismatch_condition = (
    (df['Age'] <= 21) &          # Age is 21
    ((df['Income'] > 400000) | (df['Assets'] > 20000000)) &   # Income or assets greater than 4 crore
    (df['Location'] == 'Rural')  # Location is Rural
)

# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = False
df.loc[mismatch_condition, 'Mismatched_Applicant'] = True

# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')

    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Mismatched_Applicant' in cell.column_letter:
                cell.fill = fill

print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants_Output_5.xlsx


In [38]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
 
# Read the life insurance data
file_path = 'Life_Insurance_data.xlsx'
df = pd.read_excel(file_path)
 
# Define the conditions for mismatched applicants
mismatch_condition = (
    (df['Age'] <= 21) &          # Age is 21
    ((df['Income'] > 4000000) | (df['Assets'] > 10000000)) &   # Income or assets greater than 4 crore
    (df['Location'] == 'Rural')  # Location is Rural
)
 
# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = 'No'
df.loc[mismatch_condition, 'Mismatched_Applicant'] = 'Yes'
 
# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')
 
    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Mismatched_Applicant' in cell.column_letter:
                cell.fill = fill
 
print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")

Mismatched applicants flagged and saved to Mismatched_Applicants_Output.xlsx


In [39]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the life insurance data
file_path = 'Life_Insurance_data.xlsx'
df = pd.read_excel(file_path)

# Define keywords for categorizing professions into 'White Collar' and 'Blue Collar'
white_collar_keywords = ['manager', 'engineer', 'analyst']
blue_collar_keywords = ['driver', 'technician', 'operator']

# Categorize professions into 'White Collar' and 'Blue Collar'
df['Profession_Category'] = 'Other'
df.loc[df['Profession'].str.lower().isin(white_collar_keywords), 'Profession_Category'] = 'White Collar'
df.loc[df['Profession'].str.lower().isin(blue_collar_keywords), 'Profession_Category'] = 'Blue Collar'

# Define conditions for mismatched applicants
mismatch_condition = (
    ((df['Age'] <= 21) &                                 # Age is 21
     ((df['Income'] > 4000000) | (df['Assets'] > 10000000)) &  # Income or assets greater than 4 crore
     (df['Location'] == 'Rural')) |                    # Location is Rural
    ((df['Profession_Category'] == 'Blue Collar') &      # Blue Collar profession
     (df['Income'] > 400000) &                          # Income greater than 4 lac
     (df['Assets'] > 20000000))                         # Assets greater than 2 crore
)

# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = 'No'
df.loc[mismatch_condition, 'Mismatched_Applicant'] = 'Yes'

# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output_01.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')

    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Mismatched_Applicant' in cell.column_letter:
                cell.fill = fill

print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants_Output_01.xlsx


In [44]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the life insurance data
file_path = 'Life_Insurance_data.xlsx'
df = pd.read_excel(file_path)

# Define keywords for categorizing professions into 'White Collar' and 'Blue Collar'
white_collar_keywords = ['teacher', 'lawyer', 'doctor', 'engineer', 'analyst']
blue_collar_keywords = ['driver', 'operator', 'farmer']

# Categorize professions into 'White Collar' and 'Blue Collar'
df['Profession_Category'] = 'Other'
df.loc[df['Profession'].str.lower().isin(white_collar_keywords), 'Profession_Category'] = 'White Collar'
df.loc[df['Profession'].str.lower().isin(blue_collar_keywords), 'Profession_Category'] = 'Blue Collar'

# Define conditions for mismatched applicants
mismatch_condition = (
    ((df['Age'] <= 21) &                                 # Age is 21
     ((df['Income'] > 4000000) | (df['Assets'] > 10000000)) &  # Income or assets greater than 4 crore
     (df['Location'] == 'Rural')) |                    # Location is Rural
    ((df['Profession_Category'] == 'Blue Collar') &      # Blue Collar profession
     (df['Income'] > 4000000) &                          # Income greater than 40 lac
     (df['Assets'] > 20000000))                          # Assets greater than 2 crore
)

# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = 'No'
df.loc[mismatch_condition, 'Mismatched_Applicant'] = 'Yes'

# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')

    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Mismatched_Applicant' in cell.column_letter:
                cell.fill = fill

print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants_Output.xlsx


In [52]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
 
# Read the life insurance data
file_path = 'Life_Insurance_data.xlsx'
df = pd.read_excel(file_path)
 
# Define the conditions for mismatched applicants
mismatch_condition = (
    (((df['Income'] > 4000000) | (df['Assets'] > 10000000)) & (df['Age'] <= 21)) |  # Income or assets greater than 4 crore and age less than or equal to 21
    ((df['Profession'].isin(['driver', 'technician', 'operator','farmer'])) & (df['Income'] > 4000000))|(df['Assets']> 20000000) &  # Profession is 'driver', 'technician', 'operator' and income greater than 4 crore
    (df['State'] == 'MH')  # Location is Rural
)
 
# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = "NO"
df.loc[mismatch_condition, 'Mismatched_Applicant'] = "Yes"
 
# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')
 
    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    for col in output_sheet.iter_cols():
        for cell in col:
            if 'Mismatched_Applicant' in cell.column_letter:
                cell.fill = fill
 
print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")

Mismatched applicants flagged and saved to Mismatched_Applicants_Output.xlsx


In [14]:

import pandas as pd

from openpyxl import load_workbook

from openpyxl.styles import PatternFill
 
# Read the life insurance data

file_path = 'Life_Insurance_data.xlsx'

df = pd.read_excel(file_path)
 
# Calculate Z-scores for 'Income' and 'Assets' columns

df['Z_Score_Income'] = (df['Income'] - df['Income'].mean()) / df['Income'].std()

df['Z_Score_Assets'] = (df['Assets'] - df['Assets'].mean()) / df['Assets'].std()
 
# Define the conditions for mismatched applicants

mismatch_condition = (

    (((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2)) & (df['Age'] < 21)) |  # Z-score for Income or Assets greater than 2 and Age less than 21

    ((df['Profession'].isin(['driver', 'technician', 'operator'])) & ((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2))) |  # Profession is 'driver', 'technician', 'operator' and Z-score for Income greater than 2 or Z-score for Assets greater than 2

    ((df['Age'] < 21) & (df['State'] == 'MH'))  # Age is less than 21 and Location is Rural

)
 
# Create a new column 'Mismatched_Applicant' to flag mismatched applicants

df['Mismatched_Applicant'] = False

df.loc[mismatch_condition, 'Mismatched_Applicant'] = True
 
# Save the results to a new Excel file with highlighted mismatched applicants

output_excel_file_path = 'Mismatched_Applicants_Output1.xlsx'

with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:

    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')
 
    # Highlight mismatched applicants in the output sheet

    output_sheet = writer.sheets['Mismatched_Applicants']

    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
 
    # Get the column index for 'Mismatched_Applicant'

    mismatch_column_index = df.columns.get_loc('Mismatched_Applicant') + 1
 
    for row in output_sheet.iter_rows(min_row=2, max_row=output_sheet.max_row, min_col=mismatch_column_index, max_col=mismatch_column_index):

        for cell in row:

            if cell.value:

                cell.fill = fill
 
print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants_Output1.xlsx
